# Welcome to the ProcessMCRaT Example Notebook


### This notebook will walk through some of the ways that the ProcessMCRaT library can be used to process the output of a [MCRaT](https://github.com/lazzati-astro/MCRaT) simulation and conviently get and plot results quickly.

This notebook can be run interactively on your local machine if you:
1. Install Git LFS, which is necessary to download the mcdata file from the repo that is needed for the below examples. The website to do this is [https://git-lfs.github.com/](https://git-lfs.github.com/) and the setup is very simple.
2. Clone the git repository with 
```
git clone https://github.com/parsotat/ProcessMCRaT.git
```
3. After, the notebook can be accessed by navigating to the `notebook/` directory and running 
```
jupyter notebook
```
in the command line.

Alternatively, this notebook can be run remotely and interactively by opening it in Binder at the link: [https://mybinder.org/v2/gh/parsotat/ProcessMCRaT/HEAD?filepath=notebooks%2Fprocessmcrat_example.ipynb](https://mybinder.org/v2/gh/parsotat/ProcessMCRaT/HEAD?filepath=notebooks%2Fprocessmcrat_example.ipynb)

In either case the steps below will help you get an understanding of how to setup and use the ProcessMCRaT python package.

### 1. First we need to install the package which can be done using *pip*

In the command line you would run `pip install processmcrat` or from a jupyter notebook you would run the line below:

In [1]:
!pip install processmcrat

### 2. With the package now installed on your local machine or in the Binder environment we can now import the package alongside other packages that we will need to analyze a given MCRaT simulation

In [2]:
import processmcrat as pm
import astropy.units as unit
from astropy import constants as const
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
plt.ion() #this allows matplotlib plots to show without explicit calls to plt.show

### 3. Now that we have loaded everything, we need a MCRaT simulation file to analyze. I have placed one such file in this directory from the variable *40sp_down* simulation that includes polarization and cyclo-synchrotron emission and absorption. The simulation file correponds to the last frame of the hydrodynamic simulation and we will produce mock observations from it.

In [ ]:
#here we will fownload the mcdata file needed for this example if you have not done the git lfs process or if you are 
#on Binder running the notebook
import gdown
url = 'https://drive.google.com/uc?id=12ZtMIMeLKp03Uiohfze52UBwmVJmoje0'
output = 'mcdata_1999.h5'
gdown.download(url, output, quiet=False)

We first need to create a MCRaT simulation load object that will hold all the information in the mcdata file. We pass the location of the directory that holds the simulation's mcdata files. If we dont pass any file directory then the library assumes that the mcdata file(s) are in the current working directory.

In [3]:
mcrat_sim=pm.McratSimLoad() 

if the mcdata files are located in `/Volume/dir/to/mcrat/sim`, we would instead do `pm.McratSimLoad('/Volume/dir/to/mcrat/sim')`  

Now we need to specify which frame we want to load and whether we want to load any other optional data such as stokes parameters, comoving four momenta, or the photon type. For our example analysis, the last frame of the hydro simulation is frame 1999 which is the last frame of the MCRaT simulation as well. Additionally, we only want to load in the stokes parameters to calculate polarizations later on. By leaving out the `read_comv` and `read_type` keywords, they are assumed to be `False`. We will see that by setting `read_stokes=True` the output of the functions that create mock observations will automatically include polarization calculations.

In [4]:
mcrat_sim.load_frame(1999, read_stokes=True)

The `mcrat_sim` object now holds all the information for all the MCRaT photons and various quantities can be accessed of desired. We can get one of the following quantities: 
* lab frame 4 momenta (*p0, p1, p2, p3*), 
* location of each photon in cartesian coordinates of x,y, and z (*r0, r1, r2*), 
* weights (*weight*), 
* number of scatterings experienced by each photon (*scatterings*), 
* location of the photon in the original hdf5 mcdata file (*file_index*)
* comoving fluid frame 4 momenta if loaded (*comv_p0, comv_p1, comv_p2, comv_p3*),
* photon type if loaded (*photon_type*)
* and the stokes parameters if loaded (*s0, s1, s2, s3*)

The quantities are accessed from the `mcrat_sim` object as `mcrat_sim.loaded_photons.x`, where x is any of the italicised identifiers listed above. For example, if we want to see the photon energies we can do:

In [5]:
mcrat_sim.loaded_photons.p0*(const.c.cgs.value)

array([5.27059383e-08, 1.26476577e-08, 6.22665414e-09, ...,
       2.00832919e-12, 2.74379951e-12, 6.48917431e-13])

Here we see that the loaded photons' 4-momenta are normalized by c, the speed of light in cgs units, which is how they are stored in the MCRaT simulations. We can get thier energies in erg simply by multiplying by c.

Since the energies are something that is typically analyzed, the `mcrat_sim.loaded_photons`, which is an instance of the PhotonList class, has a method associated with it that allows us to quickly calculate the energies of the photons. It can be called as:

In [6]:
mcrat_sim.loaded_photons.get_energies() #for comoving energies of the photons, the function is: get_comv_energies()

array([3.28964592e+01, 7.89404701e+00, 3.88637183e+00, ...,
       1.25350049e-03, 1.71254496e-03, 4.05022403e-04])

By default, the energy is calculated in units of keV, but by passing an applicable astropy unit to the function, the energy will be appropriately calculated. For example, to calculate the units in ergs, we do

In [7]:
mcrat_sim.loaded_photons.get_energies(unit=unit.erg)

array([5.27059383e-08, 1.26476577e-08, 6.22665414e-09, ...,
       2.00832919e-12, 2.74379951e-12, 6.48917431e-13])

If we want a wavelength, we can also do that.

In [8]:
mcrat_sim.loaded_photons.get_energies(unit=unit.angstrom)

array([1.04697493e-15, 2.51238872e-16, 1.23689113e-16, ...,
       3.98943720e-20, 5.45040918e-20, 1.28903934e-20])

#### An aside about units

This brings us to an important aside about units used in ProcessMCRaT. The package assigns appropriate astropy units to various quantities to make sure that various calculations are done consistently. It is beneficial for the user to read some of the excellent documentation that the astropy collaboration has on their implementation of units as it will help aid in understanding the ProcessMCRaT library. This documentation can be found at [https://docs.astropy.org/en/stable/units/](https://docs.astropy.org/en/stable/units/)

### 4. Conduct a Mock Observation
With a basic understanding of the McratSimLoad object and its use in loading in simulation data, we can create a mock observation with the photons. To do that we create a MockObservation object. This object, holds all the data for:
* where an observer is located with respect to the jet axis, $\theta_\mathrm{v}$, 
* the acceptance angle of the observer $\Delta \theta$, meaning that photons that are moving towards the observer in the range [$\theta_\mathrm{v}-0.5\Delta \theta, \theta_\mathrm{v}+0.5\Delta \theta$) are considered to be detected
* the location of the observer, $r_\mathrm{d}$, 
* the number of frames per second in the hydrodynamic simulation that was analyzed, $\mathrm{fps}$,
* the number of dimensions of the hydrodynamic simulation, and
* all of the data for photons detected by the observer at $\theta_\mathrm{v}$

For the *40sp_down* simulation, we would make a mock observation as:

In [9]:
observation=pm.MockObservation(1, 1, 1e13, 10, mcratsimload_obj=mcrat_sim)

where the first variable passed to the observation function is $\theta_\mathrm{v}$ in degrees, the second is $\Delta \theta$ in degrees, the third is $r_\mathrm{d}$ in cm, the fourth is fps. The last argument is the McratSimLoad object that contains all the loaded photons' data. We did not specify the number of dimensions in the call because the default is set to 2D. If the hydro simulation was in 3D, which will be supported in the future, you would have `hydrosim_dim=3` keyword as one of the arguments. 

The mock observation function call calculates which photons are moving towards the observer in the angle range specified previously. For these detected photons, their detection time, $t_d$, is calculated as:
$t_d=t_\mathrm{real} + t_p - t_j$, where $t_\mathrm{real}$ is the lab frame time of the hydrodynamic frame that has been loaded, $t_j$ is the jet detection time which is calculated by considering a virtual photon propagating to $r_\mathrm{d}$ at the time that the jet is initially launched. Finally, $t_p$ is the time that it will take photons to propagate towards $r_\mathrm{d}$ from their location in the loaded MCRaT frame.

Like the McratSimLoad object that we were exploring earlier, the MockObservation object that we created contains all the information for the detected photons. We can get all the same photon data as before with the addition of each photons' detection time. To access this data we would do:

In [10]:
observation.detected_photons.p0

array([1.10302427e-18, 1.17673392e-18, 6.61093762e-19, ...,
       5.72090955e-22, 3.51041574e-22, 1.38124376e-22])

In [11]:
observation.detected_photons.detection_time

array([11.70556666, 11.75620169, 11.68942642, ..., 39.33796243,
       39.32549413, 39.44886216])

In order to easily save our observations, we can create event files. These event files only contain information about the detected photons at the user specified $\theta_\mathrm{v}$ which allow them to be smaller than the MCRaT mcdata files that can be very large and difficult to send to collaborators, for example. These files can be loaded as an observation object, allowing for continued analysis of the data. 

In order to create an event file, we can do:

In [12]:
observation.save_event_file('40sp_down')

In [17]:
!ls

mcdata_1999.h5             processmcrat_example.ipynb


We can see that the event file has been created. It is a text file with the .evt extension. The file name contains the identifier that we passed to the `save_event_file` function followed by $r_\mathrm{d}$ and $\theta_\mathrm{v}$ for the observation. If we wanted the event file to be saved elsewhere, then we can also specify the directory using the `save_directory` optional keyword. For example, to save the file in your downloads folder you would do `observation.save_event_file('40sp_down', save_directory='~/Downloads/')`.

Using the event file, you or a collaborator can load it to create the same observation object that we created above from the McratLoadSim object. Instead of running `observation=pm.MockObservation(1, 1, 1e13, 10, mcratsimload_obj=mcrat_sim)` we would do:
```
observation=pm.MockObservation(1, 1, 1e13, 10, id='40sp_down', directory='path/to/dir/with/eventfile/')
```
where we still provide the same inputs related to the observer and the hydrosimulation, but we specify the id of the observation and the directory that the event file exists in. If the event file is in the current working directory, the directory keyword can be left out.

In [ ]:
!rm 40sp_down_1.00e+13_1.evt #since we wont be using this file in this notebook we will remove it

### 5. Create Mock Observables
Once we have conducted our mock observation we can create mock observables. These include producing light curves, spectra, and polarizations. We use the observation object directly to acquire these observables.

#### Calculating Spectra
In order to calculate time integrated spectra, from t=0 to t=45 s for our mock observed GRB, we simply do:

In [19]:
spectrum_dict=observation.spectrum(0, 45)

This returns a dictionary object that contains all of the information in the spectrum. We can see what information there is by running:

In [21]:
spectrum_dict.keys()

dict_keys(['spectrum', 'spectrum_errors', 'ph_num', 'num_scatt', 'energy_bin_center', 'theta_observer', 'pol_deg', 'stokes_i', 'stokes_q', 'stokes_u', 'stokes_v', 'pol_angle', 'pol_deg_errors', 'pol_angle_errors'])

which shows that we have the spectrum, its errors, the number of photons in each spectral bin, the center of each energy bin that was used to construct the spectra, the stokes parameters, the calculated polarization degree and angles in each energy bin and their errors, and the observer viewing angle for this mock observation. The polarization related keys are automatically included since we had set `read_stokes=True` when we read in the MCRaT data.

In order to access any of this information we simply do `spectrum_dict[X]` where X is any of the previously listed keys. For example, if we want to look at the spectrum and the energies that the spectral intensities correspond to we do

In [23]:
spectrum_dict['spectrum']

<Quantity [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           1.17822889e+38, 9.91010433e+39, 3.43779394e+40, 2.19551551e+41,
           5.70373354e+41, 9.76434454e+41, 2.13378811e+42, 6.21597438e+42,
           4.28195972e+42, 1.28857351e+43, 6.35663084e+42, 1.14929184e+43,
           1.36316567e+43

In [24]:
spectrum_dict['energy_bin_center']

<Quantity [1.12201845e-07, 1.41253754e-07, 1.77827941e-07, 2.23872114e-07,
           2.81838293e-07, 3.54813389e-07, 4.46683592e-07, 5.62341325e-07,
           7.07945784e-07, 8.91250938e-07, 1.12201845e-06, 1.41253754e-06,
           1.77827941e-06, 2.23872114e-06, 2.81838293e-06, 3.54813389e-06,
           4.46683592e-06, 5.62341325e-06, 7.07945784e-06, 8.91250938e-06,
           1.12201845e-05, 1.41253754e-05, 1.77827941e-05, 2.23872114e-05,
           2.81838293e-05, 3.54813389e-05, 4.46683592e-05, 5.62341325e-05,
           7.07945784e-05, 8.91250938e-05, 1.12201845e-04, 1.41253754e-04,
           1.77827941e-04, 2.23872114e-04, 2.81838293e-04, 3.54813389e-04,
           4.46683592e-04, 5.62341325e-04, 7.07945784e-04, 8.91250938e-04,
           1.12201845e-03, 1.41253754e-03, 1.77827941e-03, 2.23872114e-03,
           2.81838293e-03, 3.54813389e-03, 4.46683592e-03, 5.62341325e-03,
           7.07945784e-03, 8.91250938e-03, 1.12201845e-02, 1.41253754e-02,
           1.77827941e-02

We can immediately see that each of these quantities have units associated with them. The units were implicitly passed to the spectrum function when we called it. To see the defaults of th spectrum method, we can run

In [25]:
observation.spectrum?

From the docstring that pop ups, we saw that the default units of `spectrum_unit` was erg/s/keV, the energy range over which the spectrum was calculated was from [$10^{-7}$, $10^{5}$] with lograithmic step sizes of $0.1$. The default energy units are in keV, which is the normalization for the calculated spectrum. These defaults can be changed by passing in the values that we want. If we wanted the spectrum to be calculated from [$10^{2}$, $10^{8}$] eV with units of counts/s/eV then we set `spectrum_unit=unit.count/unit.s/unit.eV`, `log_energy_range=[2, 8]`, and `energy_unit=unit.eV`. This would look like:

In [27]:
spect_count_dict=observation.spectrum(0, 45, spectrum_unit=unit.count/unit.s/unit.eV, energy_unit=unit.eV,\
                                      log_energy_range=[2, 8])
print(spect_count_dict['spectrum'], spect_count_dict['energy_bin_center'])

[8.16839675e+49 6.66499920e+49 1.10588531e+50 2.98814821e+50
 5.19895879e+49 1.55375944e+50 1.28119756e+50 4.16988062e+50
 3.18443318e+50 9.99782387e+50 9.18733123e+50 1.32534586e+51
 3.07912512e+51 5.01825368e+51 7.57730005e+51 6.24319146e+51
 8.09064495e+51 9.14677899e+51 1.28162830e+52 1.04730973e+52
 1.06038295e+52 1.12076214e+52 1.08074270e+52 1.03032663e+52
 1.04691193e+52 1.11913352e+52 9.68918762e+51 8.41355845e+51
 7.45050964e+51 6.20296464e+51 5.10983657e+51 4.08418133e+51
 2.94378854e+51 2.10446969e+51 1.41316825e+51 8.50736483e+50
 4.75384091e+50 2.48670239e+50 1.34067593e+50 6.39102053e+49
 3.31256265e+49 1.86968575e+49 9.45528143e+48 5.42084507e+48
 2.63690364e+48 1.12484082e+48 5.16231681e+47 2.52951898e+47
 1.20511077e+47 9.28059451e+46 4.34490182e+46 1.55694072e+46
 1.13923016e+46 5.51659221e+45 8.45644861e+44 6.10654172e+44
 4.85059851e+44 1.92648367e+44 0.00000000e+00 0.00000000e+00] ct / (eV s) [1.12201845e+02 1.41253754e+02 1.77827941e+02 2.23872114e+02
 2.81838293

The times that are passed into the spectrum method can correspond to any times within the active period of the mock observed GRB which allows us to also create mock observed time resolved spectra. 


One of the most important things that is done with time resolved and time integrated spectra is fitting them with models (the code is only able to fit spectra with the Band and Comptonized functions at this time). **In order to do this, we have to give some information related to the fitting of the spectra. One this is done, we call the spectrum method where the units of the spectra need to be in counts/s/energy_unit. We also need to let the spectrum function that we want to do the spectral fitting.** To get a spectrum and its fitted Band or Comptonized function parameters we do:

In [29]:
observation.set_spectral_fit_parameters()

By running the code below, we can see that we just initalized the spectral energy range that we want the spectral fitting to occur over to be from 8 to 40000 keV. We also specified that we want to fit any energy bins that have greater than 10 photons in them, as that will allow us to assume that the errors in those bins are gaussian for the fitting technique used here. If you would like to change any of these parameters simply set each keyword to the values that you want when calling the `set_spectral_fit_parameters` method. For example, if we wanted to fit spectra from 8-300 eV, with either the Band or Comptonized functions, for energy bins with at least 20 photons in them we would do `observation.set_spectral_fit_parameters(spectral_fit_energy_range=[8,300], spectral_fit_energy_unit=unit.eV, approx_gaussian_error_num=20)`

In [32]:
observation.set_spectral_fit_parameters?

In [33]:
spectrum_dict=observation.spectrum(0, 45, spectrum_unit=unit.count/unit.s/unit.keV, fit_spectrum=True, sample_num=1e4)

/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:620: RuntimeWarning: invalid value encountered in sqrt
  sigma_Q_r = np.sqrt(var_factor * (mu_factor / mu - Q_r ** 2))
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:639: RuntimeWarning: invalid value encountered in sqrt
  partial_pr_Ur * sigma_U_r) ** 2 + 2 * partial_pr_Qr * partial_pr_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:641: RuntimeWarning: invalid value encountered in sqrt
  partial_phir_Ur * sigma_U_r) ** 2 + 2 * partial_phir_Qr * partial_phir_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mclib.py:41: RuntimeWarning: invalid value encountered in double_scalars
  model[kk]=((alpha-beta)*break_energy)**(alpha-beta)*energies[kk]**(beta)*np.exp(beta-alpha)


In [34]:
spectrum_dict.keys()

dict_keys(['spectrum', 'spectrum_errors', 'ph_num', 'num_scatt', 'energy_bin_center', 'theta_observer', 'pol_deg', 'stokes_i', 'stokes_q', 'stokes_u', 'stokes_v', 'pol_angle', 'pol_deg_errors', 'pol_angle_errors', 'fit', 'fit_errors', 'model_use'])

We can now see that there are new keys for the spectral fit, the errors in the spectral fitted parameters, and the type of model that provides the superior fit (either 'b' for Band or 'c' for Comptonized function, as of now). As is shown below, the value for the 'fit' key gives another dictionary with the fitted $\alpha$, $\beta$ (if applicable), break energy, and normalizations. The value for the 'fit_errors' key gives another dictionary with the fitted $\alpha$, $\beta$ (if applicable), and break energy $1\sigma$ errors that have been acquired by bootstrapping a number of times corresponding to the `sample_num` keyword that was passed to the spectrum method.

In [41]:
print(spectrum_dict['fit'], spectrum_dict['fit_errors'])
print(r'The best fit spectrum is a', spectrum_dict['model_use'], r'The best fit $\alpha$ is ',\
      spectrum_dict['fit']['alpha'], 'with an error of', spectrum_dict['fit_errors']['alpha_errors'])

{'alpha': -0.011663412596284919, 'beta': -2.820302047290041, 'break_energy': <Quantity 131.70964468 keV>, 'normalization': 1.5901140369751522e+57} {'alpha_errors': 0.004104947209720429, 'beta_errors': 0.006513225094375061, 'break_energy_errors': <Quantity 0.78650231 keV>}
The best fit spectrum is a b The best fit $\alpha$ is  -0.011663412596284919 with an error of 0.004104947209720429


In order to calculate the spectral peak energy, we can use the included `calc_epk_error` function. For example:

In [55]:
peak_e, peak_e_err = pm.calc_epk_error(spectrum_dict['fit']['alpha'], spectrum_dict['fit']['break_energy'],\
                                        alpha_error=spectrum_dict['fit_errors']['alpha_errors'],\
                                        break_energy_error=spectrum_dict['fit_errors']['break_energy_errors'])
print(peak_e, peak_e_err)

261.8831054281589 keV 1.654654906211803 keV


The last keyword that can be passed to the spectrum method that has not been discussed as of yet is the `photon_type`. As a default is set to `None` which means that all types of photons are included in the construction of the spectrum, both 'i' and 's' types ([see Section 3.3 of the MCRaT documentation](https://github.com/lazzati-astro/MCRaT/blob/master/Doc/mcrat_doc.pdf)) for injected photons and cyclo-synchrotron photons. If the user wants to analyze either of these populations of photons they can set `photon_type='i'` or `photon_type='s'`.

If you want to use different parameters for calculating spectra later on in your analysis, you can simply recall the observation.set_spectral_fit_parameters with new values for spectral_fit_energy_range, spectral_fit_energy_unit, and/or approx_gaussian_error_num.

We have also included conenience functions to calculate the Band and Comptonized functions for a given set of parameters, thus the results of the spectral fits can be sent to these functions with energy ranges and you will be able to reconstruct the best fit spectrum. More information can be found by doing:

In [68]:
pm.band_function?

In [ ]:
pm.comptonized_function?

#### Calculating Polarization

In order to calculate the time integrated polarization, we can simply do:

In [42]:
time_int_pol=observation.polarization(0,45)

This gives us another dictionary with a subset of the keys that the spectrum method provided. As is shown below, we get the stokes parameters, the polarization degree and angles, as well as the $1\sigma$ errors of those mock observations.

In [43]:
time_int_pol.keys()

dict_keys(['stokes_i', 'stokes_q', 'stokes_u', 'stokes_v', 'pol_deg', 'pol_angle', 'pol_deg_errors', 'pol_angle_errors', 'theta_observer'])

Time resolved polarizations can be calculated simply by changing the time bins that the photons are collected within. If we want to calculate the polarization for a given type of photon in the simulation, we can also pass the `photon_type` keyword into our method call, the same as what we would do for calculating a spectrum.

Similar to the way that we can specify an energy range to calculate the spectra within, we can also specify an energy range over which the polarization is calculated. This is done by setting the `energy_range` and `energy_unit` parameters when calling the `polarization` method. For example, if we wanted to calculated the time integrated polarization in the optical enegies from 1.5-7.7 eV then we would do:

In [44]:
observation.polarization(0,45, energy_range=[1.5,7.7], energy_unit=unit.eV)

{'stokes_i': <Quantity 1.>,
 'stokes_q': <Quantity -0.25713576>,
 'stokes_u': <Quantity -0.01705302>,
 'stokes_v': <Quantity 0.>,
 'pol_deg': <Quantity 0.25770061>,
 'pol_angle': <Quantity -88.10287526 deg>,
 'pol_deg_errors': <Quantity 0.17614188>,
 'pol_angle_errors': <Quantity 21.0030049 deg>,
 'theta_observer': <Quantity 1. deg>}

#### Calculating Light Curves

Calculating light curves is just as easy as calculating spectra and polarization. To get the light curve that an observer located at $\theta_\mathrm{v}=1^\circ$ would detect from our *40sp_down* simulated GRB, from 10 s after the jet was launched to 40 s after the jet was launched in uniform time bins, we would do:

In [47]:
lightcurve_dict=observation.lightcurve(time_start=10, time_end=40, dt=0.2)
lightcurve_dict.keys()

/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:639: RuntimeWarning: invalid value encountered in sqrt
  partial_pr_Ur * sigma_U_r) ** 2 + 2 * partial_pr_Qr * partial_pr_Ur * cov)


dict_keys(['lightcurve', 'lightcurve_errors', 'ph_num', 'num_scatt', 'times', 'theta_observer', 'pol_deg', 'stokes_i', 'stokes_q', 'stokes_u', 'stokes_v', 'pol_angle', 'pol_deg_errors', 'pol_angle_errors'])

Looking at the various keys in the dictionary we can see that we get similar keys as from the previous dictionaries. The difference here is that the output quantities are functions of time. The times that we get define the time bins that photons are collected within to produce the lightcurves, polarization, and other quantities. 

If we wanted to produce the light curve using variable time bins, we can set `variable_t_bins=True` such as:

In [48]:
lightcurve_dict=observation.lightcurve(time_start=10, time_end=40, dt=0.2, variable_t_bins=True)

/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:639: RuntimeWarning: invalid value encountered in sqrt
  partial_pr_Ur * sigma_U_r) ** 2 + 2 * partial_pr_Qr * partial_pr_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:641: RuntimeWarning: invalid value encountered in sqrt
  partial_phir_Ur * sigma_U_r) ** 2 + 2 * partial_phir_Qr * partial_phir_Ur * cov)


Alteratively, if you have an array that defines the time bins that you would like the light curve and other quantities to be calculated from, you can pass that information instead as:

In [53]:
t=np.arange(10, 40, 2)
lightcurve_dict=observation.lightcurve(time_array=t)
print(lightcurve_dict['times'])
print(lightcurve_dict['lightcurve'])

[10. 12. 14. 16. 18. 20. 22. 24. 26. 28. 30. 32. 34. 36. 38.] s
[2.58500125e+50 1.21760483e+51 7.49266623e+51 1.63348443e+52
 2.62399980e+52 2.00367341e+52 4.37300696e+51 3.50573198e+50
 4.69518325e+51 1.71027976e+52 3.34085955e+51 1.81023401e+51
 7.53685977e+51 8.58960867e+51 0.00000000e+00] erg / s


We can see that the times have been used to calculate the lightcurve. The default calculation of the lightcurve produces values that have units of erg/s however the light curve can also be set to counts/s by setting `lc_unit=unit.count/unit.s`. These light curves were calculated without any energy cuts making them bolometric light curves however, we can also calculate the light curve for a specific energy range of photons. For example if we wanted to calculate the light curve from 10-1000 eV we would do something like:

In [56]:
lightcurve_dict=observation.lightcurve(time_start=10, time_end=40, dt=0.2, energy_range=[10, 1000],\
                                       energy_unit=unit.eV)

/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:639: RuntimeWarning: invalid value encountered in sqrt
  partial_pr_Ur * sigma_U_r) ** 2 + 2 * partial_pr_Qr * partial_pr_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:641: RuntimeWarning: invalid value encountered in sqrt
  partial_phir_Ur * sigma_U_r) ** 2 + 2 * partial_phir_Qr * partial_phir_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:621: RuntimeWarning: invalid value encountered in sqrt
  sigma_U_r = np.sqrt(var_factor * (mu_factor / mu - U_r ** 2))
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:620: RuntimeWarning: invalid value encountered in sqrt
  sigma_Q_r = np.sqrt(var_factor * (mu_factor / mu - Q_r ** 2))


For each time bin we can also calculate the time resolved spectrum and fit the spectra from each time bin. In order to do this, we set `fit_spectrum=True`. This can take a little while to calculate based on the `spectral_sample_num` value. This is identical to the `sample_num` keyword that gets passed to the `spectrum` method. To minimize the wait time in this notebook, we have decreased the number of time bins that will have spectra that need to be fitted.

In [57]:
lightcurve_dict=observation.lightcurve(time_start=10, time_end=40, dt=1, fit_spectrum=True)

Fitting between times: 10 11


/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:639: RuntimeWarning: invalid value encountered in sqrt
  partial_pr_Ur * sigma_U_r) ** 2 + 2 * partial_pr_Qr * partial_pr_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:641: RuntimeWarning: invalid value encountered in sqrt
  partial_phir_Ur * sigma_U_r) ** 2 + 2 * partial_phir_Qr * partial_phir_Ur * cov)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:620: RuntimeWarning: invalid value encountered in sqrt
  sigma_Q_r = np.sqrt(var_factor * (mu_factor / mu - Q_r ** 2))
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mockobservations.py:621: RuntimeWarning: invalid value encountered in sqrt
  sigma_U_r = np.sqrt(var_factor * (mu_factor / mu - U_r ** 2))


Fitting between times: 11 12


/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mclib.py:41: RuntimeWarning: invalid value encountered in double_scalars
  model[kk]=((alpha-beta)*break_energy)**(alpha-beta)*energies[kk]**(beta)*np.exp(beta-alpha)
/Users/Tylerparsotan/.local/lib/python3.7/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mclib.py:86: RuntimeWarning: overflow encountered in power
  model=(energies**alpha)*np.exp(-energies/break_energy)
/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mclib.py:87: RuntimeWarning: invalid value encountered in true_divide
  model=model/np.trapz(model,x=energies)*normalization


Fitting between times: 12 13
Fitting between times: 13 14
Fitting between times: 14 15
Fitting between times: 15 16
Fitting between times: 16 17
Fitting between times: 17 18
Fitting between times: 18 19
Fitting between times: 19 20
Fitting between times: 20 21
Fitting between times: 21 22
Fitting between times: 22 23
Fitting between times: 23 24
Fitting between times: 24 25
Fitting between times: 25 26
Fitting between times: 26 27
Fitting between times: 27 28
Fitting between times: 28 29
Fitting between times: 29 30
Fitting between times: 30 31
Fitting between times: 31 32
Fitting between times: 32 33
Fitting between times: 33 34
Fitting between times: 34 35
Fitting between times: 35 36
Fitting between times: 36 37
Fitting between times: 37 38
Fitting between times: 38 39


In [58]:
lightcurve_dict.keys()

dict_keys(['lightcurve', 'lightcurve_errors', 'ph_num', 'num_scatt', 'times', 'theta_observer', 'pol_deg', 'stokes_i', 'stokes_q', 'stokes_u', 'stokes_v', 'pol_angle', 'pol_deg_errors', 'pol_angle_errors', 'fit', 'fit_errors', 'model_use'])

We can now see that there are new keys in the returned dictionary related to the spectral fittings. We can access the fitted alphas by doing:

In [60]:
lightcurve_dict['fit']['alpha']

array([            nan,             nan,  1.96436731e+00,  1.05351393e+00,
        2.13555153e-02,  7.40774498e-01,  1.69985252e-01,  3.51816904e-01,
        3.45933991e-01,  4.96825873e-01,  3.34332990e-01,  9.00424439e-01,
       -2.25999481e-01, -1.18651430e-03, -3.73186133e-02,  6.68327809e-01,
        1.64518785e+00,  8.74747720e-02,  5.81461438e-01,  5.58506902e-01,
        5.87267563e-01,  9.70184684e-01, -3.04516724e-01,  6.47989946e-01,
        3.17196374e-01,  7.27955431e-01,  6.96803379e-01,  3.86126574e-01,
        7.13077558e-01,             nan])

With producing these mock observables, it becomes tedious to constantly enter a certain spectral energy range that you would like the spectra to be calculated within, or light curve energy range, or detected polarization energy range to simulate what energy ranges a given instrument would make its measurements in. In order to simplify this, the ProcessMCRaT code allows you to create an instrument with these parameters defined. Then, we can load this instrument into each of our observation objects which will automatically apply the instrumental energy constraints for each mock observable that is produced. 

Lets create a Polar-2/Fermi mash-up instrument, which would measure polarization from 20-800 keV, spectra from 8 keV-40 MeV, and light curves from 20-800 keV as well:

In [61]:
inst=pm.Instrument(name='Fermi+Polar', polarization_energy_range=[20,800], polarization_energy_unit=unit.keV,\
                  spectral_energy_range=[8,40000], spectral_energy_unit=unit.keV, lightcurve_energy_range=[20,800],\
                   lightcurve_energy_unit=unit.keV)

If any of the fields are not specified and left off of the call to create the instrument, then they will be ignored and the calculations for those mock observables will default to being integrated over energy. For example if we did `pm.Instrument(name='Fermi+Polar', polarization_energy_range=[20,800], polarization_energy_unit=unit.keV, spectral_energy_range=[8,40000], spectral_energy_unit=unit.keV` then the light cuves that we could calculate after loading the instruent into our observation object would be bolometric light curves. If we left off `spectral_energy_range=[8,40000]` and `spectral_energy_unit=unit.keV` then the spectra would be calculted for the default energy ranges in the spectrum method.

Now that we have our instrument, we can tell our observation object that we want to use this instrument by loading it with:

In [65]:
observation.load_instrument(inst)

With the instrument loaded we can recalcalculate some of our mock observables. 

In [66]:
inst_spec_dict=observation.spectrum(0, 45, spectrum_unit=unit.count/unit.s/unit.keV, fit_spectrum=True, sample_num=1e4)

/opt/anaconda3/lib/python3.7/site-packages/processmcrat/mclib.py:41: RuntimeWarning: invalid value encountered in double_scalars
  model[kk]=((alpha-beta)*break_energy)**(alpha-beta)*energies[kk]**(beta)*np.exp(beta-alpha)


The above call calculates our time integrated light curve from 8 keV-40 MeV and it fits our spectrum from 8 keV-40 MeV due to the fact that we specified this energy range to conduct our spectral fitting within when we ran `observation.set_spectral_fit_parameters()`.

After calculating the light curves, polarization, and spectra using the instrument, you can unload i from the observation by doing:

In [64]:
observation.unload_instrument()

If, while having the instrument loaded, you want to explore what a given mock obervable would look like in a different energy range, you can specify that in the method call and it will override the instrumental energy constraints.

### 6. Plotting Mock Observables

Now that we have explored some of the mock observables that can be created with a given MockObservable object, we can plot some of the results